# Primetrade.ai Data Science Internship Assignment

## Task: analyze historical trade data from various Binance accounts over 90 days and calculate inancial metrics for each account, rank them, and provide a top 20 list.

### Metrics to Calculate:
- ROI (Return on Investment)
- PnL (Profit and Loss)
- Sharpe Ratio
- MDD (Maximum Drawdown)
- Win Rate
- Win Positions
- Total Positions

## Steps to Perform in this assignment:
### 1. Data Exploration and Cleaning:

- Load and inspect the dataset, handle missing values.

### 2. Feature Engineering:

- Determine feature importance and create a scoring system with weighted scores.


### 3. Ranking Algorithm:

- Develop an algorithm to rank accounts based on calculated metrics.

### 4. Documentation:

- Provide a concise report on methodology, findings, and assumptions.

## Deliverables:

- Jupyter Notebook or Python script with complete analysis and code.
- CSV file containing calculated metrics.
- List of top 20 accounts based on ranking.
- Report detailing approach and findings.

### -----------------------------------------------------------------------------------------

### Before we start with the steps mentioned above, lets import the libraries that we will be requiring to finish this task



In [1]:
import pandas as pd
import numpy as np
import ast

### Now onto the first step

## 1. Data Exploration and Cleaning

In [2]:
file_path = '/Users/varunv/Desktop/Primetrade.ai assessment/TRADES_CopyTr_90D_ROI.csv' # Loading the Dataset
data = pd.read_csv(file_path)

In [3]:
# Data Inspection
print(data.info())
print(data.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB
None
              Port_IDs                                      Trade_History
0  3925368433214965504  [{'time': 1718899656000, 'symbol': 'SOLUSDT', ...
1  4002413037164645377  [{'time': 1718980078000, 'symbol': 'NEARUSDT',...
2  3923766029921022977  [{'time': 1718677164000, 'symbol': 'ETHUSDT', ...
3  3994879592543698688  [{'time': 1718678214000, 'symbol': 'ETHUSDT', ...
4  3926423286576838657  [{'time': 1718979615000, 'symbol': 'ETHUSDT', ...


In [4]:
# Drop rows where Trade_History is missing
data_cleaned = data.dropna(subset=['Trade_History']).copy()

### Now we can see that the Trade_History column contains all the information that we require but in order to access it properly, we will have to extract information and store it in an expanded dataframe.

In [5]:
# Parse the Trade_History column into a structured format
def parse_trade_history(trade_history):
    try:
        return ast.literal_eval(trade_history)
    except (ValueError, SyntaxError):
        return None

    
data_cleaned['Parsed_Trade_History'] = data_cleaned['Trade_History'].apply(parse_trade_history)

In [6]:
# Expand Parsed_Trade_History into a flat dataframe
expanded_rows = []

for _, row in data_cleaned.iterrows():
    port_id = row['Port_IDs']
    trades = row['Parsed_Trade_History']
    if isinstance(trades, list):
        for trade in trades:
            trade['Port_ID'] = port_id
            expanded_rows.append(trade)


### Now we will create a new dataframe from the expanded rows

In [7]:
# Create a new dataframe from the expanded rows
expanded_data = pd.DataFrame(expanded_rows)

# Inspect the resulting dataframe structure
expanded_data.info(), expanded_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 211277 entries, 0 to 211276
Data columns (total 15 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   time                 211277 non-null  int64  
 1   symbol               211277 non-null  object 
 2   side                 211277 non-null  object 
 3   price                211277 non-null  float64
 4   fee                  211277 non-null  float64
 5   feeAsset             211277 non-null  object 
 6   quantity             211277 non-null  float64
 7   quantityAsset        211277 non-null  object 
 8   realizedProfit       211277 non-null  float64
 9   realizedProfitAsset  211277 non-null  object 
 10  baseAsset            211277 non-null  object 
 11  qty                  211277 non-null  float64
 12  positionSide         211277 non-null  object 
 13  activeBuy            211277 non-null  bool   
 14  Port_ID              211277 non-null  int64  
dtypes: bool(1), float

(None,
             time    symbol side      price       fee feeAsset    quantity  \
 0  1718899656000   SOLUSDT  BUY  132.53700 -0.994027     USDT  1988.05500   
 1  1718899618000  DOGEUSDT  BUY    0.12182 -0.279796     USDT  1398.98088   
 2  1718899618000  DOGEUSDT  BUY    0.12182 -0.039494     USDT   197.47022   
 3  1718899616000  DOGEUSDT  BUY    0.12182 -0.008284     USDT    16.56752   
 4  1718899616000  DOGEUSDT  BUY    0.12182 -0.046109     USDT    92.21774   
 
   quantityAsset  realizedProfit realizedProfitAsset baseAsset      qty  \
 0          USDT             0.0                USDT       SOL     15.0   
 1          USDT             0.0                USDT      DOGE  11484.0   
 2          USDT             0.0                USDT      DOGE   1621.0   
 3          USDT             0.0                USDT      DOGE    136.0   
 4          USDT             0.0                USDT      DOGE    757.0   
 
   positionSide  activeBuy              Port_ID  
 0         LONG      

### we can now see that all the different values stored in the Trade_History column have been extracted and stored into different columns in this new dataframe that we have created. 

### Now we move onto the Feature Engineering step

## 2. Feature Engineering

### In this step we will be ensuring that all the information that we need is present in the dataframe that we created in the datatype that we require. 
### We will also be calculating different metrics and create a scoring system with weighted scores

In [8]:
# Ensure expanded_data contains necessary fields
expanded_data['timestamp'] = pd.to_datetime(expanded_data['time'])
expanded_data['realizedProfit'] = expanded_data['realizedProfit'].astype(float)
expanded_data['quantity'] = expanded_data['quantity'].astype(float)

In [9]:
# Group data by Port_ID
grouped = expanded_data.groupby('Port_ID')

In [10]:
# Calculate ROI
def calculate_roi(group):
    initial_investment = group['quantity'].iloc[0]  # Assuming the first quantity as the initial investment
    net_profit = group['realizedProfit'].sum()
    return (net_profit / initial_investment) * 100 if initial_investment != 0 else 0

roi = grouped.apply(calculate_roi)

In [11]:
pnl = grouped['realizedProfit'].sum()

In [12]:
# Calculate Sharpe Ratio
def calculate_sharpe(group):
    mean_returns = group['realizedProfit'].mean()
    std_returns = group['realizedProfit'].std()
    risk_free_rate = 0.01  # Assumed constant
    return (mean_returns - risk_free_rate) / std_returns if std_returns != 0 else 0

sharpe_ratio = grouped.apply(calculate_sharpe)

In [13]:
# Calculate Maximum Drawdown (MDD)
def calculate_mdd(group):
    cumulative_returns = group['realizedProfit'].cumsum()
    drawdowns = cumulative_returns - cumulative_returns.cummax()
    return drawdowns.min()

mdd = grouped.apply(calculate_mdd)

In [14]:
# Calculate Win Rate
def calculate_win_rate(group):
    win_positions = (group['realizedProfit'] > 0).sum()
    total_positions = len(group)
    return (win_positions / total_positions) * 100 if total_positions != 0 else 0

win_rate = grouped.apply(calculate_win_rate)

In [15]:
# Combine metrics into a single DataFrame
metrics = pd.DataFrame({
    'ROI': roi,
    'PnL': pnl,
    'Sharpe_Ratio': sharpe_ratio,
    'MDD': mdd,
    'Win_Rate': win_rate
}).reset_index()

### Now that we are done with calculating the metrics and creating the weighted scoring system we will move on to ranking the accounts based on the calculated metrics, which is our 3rd step.

## 3. Ranking algorithm

### The ranking algorithm is as follows:|

1. Select Metrics to Use: we will use the calculated metrics:
- ROI (Return on Investment)
- PnL (Profit and Loss)
- Sharpe Ratio
- Maximum Drawdown (MDD)
- Win Rate

2. Normalize Metrics: Normalize the metrics to bring them onto a similar scale, typically between 0 and 1. Use min-max normalization
- Normalized Value = Value − Min(Value) / Max(Value) − Min(Value)

3. Assign Weights: Assign weights to the normalized metrics based on their importance to the overall ranking. For example:
- ROI: 30%
- Sharpe Ratio: 25%
- MDD: 20%
- Win Rate: 15%
- PnL: 10%

4. Calculate Weighted Score: Combine the normalized metrics using their respective weights:
Score = ∑(Normalized Metric × Weight)

5. Rank Accounts: Sort accounts by their calculated score in descending order.

6. Output Top 20 Accounts: Extract the top 20 accounts based on the score.

### Since we have already calculated the metrics, we will move on to the 2nd step which is to normalize the metrics

In [16]:
# Normalize metrics
for column in ['ROI', 'PnL', 'Sharpe_Ratio', 'MDD', 'Win_Rate']:
    metrics[f'{column}_normalized'] = (
        (metrics[column] - metrics[column].min()) /
        (metrics[column].max() - metrics[column].min())
    )

### Now we will assign weights to the normalized metrics and calculate the weighted score which is our 3rd and 4th step

In [17]:
# Weighted scoring
weights = {
    'ROI_normalized': 0.3,
    'Sharpe_Ratio_normalized': 0.25,
    'MDD_normalized': 0.2,
    'Win_Rate_normalized': 0.15,
    'PnL_normalized': 0.1
}

# Calculate weighted score
metrics['score'] = sum(metrics[f'{metric}'] * weight for metric, weight in weights.items())

### Now for the 5th step, we will be ranking the accounts by their calculated score in descending order.

In [18]:
# Rank accounts
metrics_sorted = metrics.sort_values(by='score', ascending=False)

### For the 6th and the final step of ranking algorithm we will be extracting the top 20 accounts based on their scores

In [19]:
# Extract top 20 accounts
top_20 = metrics_sorted.head(20)

## Now that we are done with all the major steps, we will now move on to creating the deliverables required in this task i.e., CSV file containing calculated metrics and a list of top 20 accounts based on ranking.

In [20]:
# Output results
metrics_sorted.to_csv("output_with_ranking.csv", index=False)
top_20.to_csv("top_20_accounts.csv", index=False)

In [21]:
print("Ranking complete. Files saved:")
print("All Metrics with Scores: output_with_ranking.csv")
print("Top 20 Accounts: top_20_accounts.csv")

Ranking complete. Files saved:
All Metrics with Scores: output_with_ranking.csv
Top 20 Accounts: top_20_accounts.csv
